<a href="https://colab.research.google.com/github/09eesx/forensic-emotion-ai/blob/main/data_cleaningandtokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn
!pip install accelerate -U
!pip install -U transformers


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import random
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import hstack  # To combine sparse matrices
from wordcloud import WordCloud

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")



In [ ]:
ana_klasor = "/content/drive/MyDrive/data/raw_data/data_aiagent/raw_data/data_aiagent/doic_elli"
etiket_dosyasi = "/content/drive/MyDrive/data/raw_data/data_aiagent/raw_data/data_aiagent/doic_elli/train_split_Depression_AVEC2017 (2).csv"

In [ ]:
# Etiketleri oku
etiket_df = pd.read_csv(etiket_dosyasi)
etiket_df["Participant_ID"] = etiket_df["Participant_ID"].astype(int)

tum_veriler = []

for dosya_adi in os.listdir(ana_klasor):
    if not dosya_adi.endswith("_TRANSCRIPT.csv"):
        continue

    try:
        participant_id = int(dosya_adi.split("_")[0])
        transcript_yolu = os.path.join(ana_klasor, dosya_adi)

        df = pd.read_csv(transcript_yolu, sep="\t", header=0)

        if "speaker" not in df.columns or "value" not in df.columns:
            print(f"{participant_id} → Geçersiz format (eksik sütun)")
            continue

        df_participant = df[df["speaker"] == "Participant"]
        df_participant = df_participant[~df_participant["value"].astype(str).str.contains("scrubbed", na=False)]
        df_participant = df_participant.dropna(subset=["value"])

        full_text = " ".join(df_participant["value"].tolist())

        etiket_satiri = etiket_df[etiket_df["Participant_ID"] == participant_id]
        if etiket_satiri.empty:
            print(f"{participant_id} → Etiket bulunamadı")
            continue

        phq_score = etiket_satiri["PHQ8_Score"].values[0]
        depresyon_etiketi = etiket_satiri["PHQ8_Binary"].values[0]

        tum_veriler.append({
            "participant_id": participant_id,
            "full_text": full_text,
            "phq_score": phq_score,
            "depression_label": depresyon_etiketi
        })

    except Exception as e:
        print(f"{dosya_adi} → Hata: {e}")

# Kaydet
veri_df = pd.DataFrame(tum_veriler)
veri_df.to_csv("cleaned_daic_woz_texts.csv", index=False)

print("Tamamlandı. Toplam katılımcı:", len(veri_df))

300 → Etiket bulunamadı
301 → Etiket bulunamadı
302 → Etiket bulunamadı
308 → Etiket bulunamadı
309 → Etiket bulunamadı
311 → Etiket bulunamadı
323 → Etiket bulunamadı
329 → Etiket bulunamadı
332 → Etiket bulunamadı
334 → Etiket bulunamadı
335 → Etiket bulunamadı
337 → Etiket bulunamadı
346 → Etiket bulunamadı
✅ Tamamlandı. Toplam katılımcı: 27


In [ ]:
import pandas as pd

# Ortak etiketler
LABELS = [
    "depresyon", "intihar eğilimi", "anksiyete", "stres", "kişilik bozukluğu", "bipolar",
    "öfke", "hakaret", "tehdit", "nefret söylemi", "üzgünlük", "umutlu", "sevinç", "normal"
]

def load_empathetic(path):
    mapping = {
        "sad": "üzgünlük",
        "ashamed": "üzgünlük",
        "disappointed": "üzgünlük",
        "angry": "öfke",
        "furious": "öfke",
        "anxious": "anksiyete",
        "afraid": "anksiyete",
        "hopeful": "umutlu",
        "trusting": "umutlu",
        "joyful": "sevinç",
        "proud": "sevinç"
    }
    df = pd.read_csv(path, on_bad_lines='skip', engine='python')
    df = df.rename(columns={"utterance": "text", "context": "original_label"})
    df["label"] = df["original_label"].map(mapping)
    df = df[["text", "label"]].dropna()
    return df


# 2. Jigsaw
def load_jigsaw(path):
    df_jigsaw = pd.read_csv(path, on_bad_lines='skip', engine='python')
    def map_jigsaw(row):
        if row["threat"] == 1:
            return "tehdit"
        elif row["toxic"] == 1:
            return "öfke"
        elif row["insult"] == 1:
            return "hakaret"
        elif row["identity_hate"] == 1:
            return "nefret söylemi"
        else:
            return None
    df_jigsaw["label"] = df_jigsaw.apply(map_jigsaw, axis=1)
    df_jigsaw = df_jigsaw.rename(columns={"comment_text": "text"})
    df_jigsaw = df_jigsaw[["text", "label"]].dropna()
    return df_jigsaw

# 3. Combined_data
def load_combined(path):
    df_comb = pd.read_csv(path)
    mapping = {
        "Anxiety": "anksiyete",
        "Normal": "normal",
        "Depression": "depresyon",
        "Suicidal": "intihar eğilimi",
        "Stress": "stres",
        "Bipolar": "bipolar",
        "Personality disorder": "kişilik bozukluğu"
    }
    df_comb = df_comb.rename(columns={"status": "original_label", "statement": "text"})
    df_comb["label"] = df_comb["original_label"].map(mapping)
    df_comb = df_comb[["text", "label"]].dropna()
    return df_comb

# 4. DAIC-WOZ
def load_daic(path):
    df_daic = pd.read_csv(path)
    df_daic["label"] = df_daic["depression_label"].apply(lambda x: "depresyon" if x == 1 else "normal")
    df_daic = df_daic.rename(columns={"full_text": "text"})
    df_daic = df_daic[["text", "label"]].dropna()
    return df_daic

# Tüm veri setlerini oku
empathetic_df = load_empathetic("/content/drive/MyDrive/data/raw_data/data_aiagent/raw_data/empatheticdialogues/empatheticdialogues/train.csv")
jigsaw_df = load_jigsaw("/content/drive/MyDrive/data/raw_data/data_aiagent/raw_data/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv")
combined_df = load_combined("/content/drive/MyDrive/data/raw_data/data_aiagent/raw_data/data_aiagent/Combined Data.csv")
daic_df = load_daic("/content/cleaned_daic_woz_texts.csv")

# Hepsini birleştir
all_data = pd.concat([empathetic_df, jigsaw_df, combined_df, daic_df], ignore_index=True)

# Geçerli etiketler dışındakileri at
all_data = all_data[all_data["label"].isin(LABELS)]



# Kaydet
all_data.to_csv("/content/drive/MyDrive/data/train_cleaned2.csv", index=False)
print(f"✅ Toplam örnek sayısı: {len(all_data)}")
print("📁 Kaydedildi: datasets/train_cleaned.csv")
print("🎯 Etiket Dağılımı:")
print(all_data['label'].value_counts())


✅ Toplam örnek sayısı: 95076
📁 Kaydedildi: datasets/train_cleaned.csv
🎯 Etiket Dağılımı:
label
öfke                 19961
normal               16360
depresyon            15414
intihar eğilimi      10652
anksiyete             8798
üzgünlük              6947
sevinç                5050
umutlu                4390
bipolar               2777
stres                 2587
kişilik bozukluğu     1077
hakaret                528
tehdit                 478
nefret söylemi          57
Name: count, dtype: int64


##oversampling

In [ ]:
# 🔹 2. Manuel oversampling (dengesiz sınıflar)
print(" Sınıf sayıları (önce):")
print(all_data["label"].value_counts())

oversample_labels = ["kişilik bozukluğu","hakaret","tehdit"]
for label in oversample_labels:
    target_df = all_data[all_data["label"] == label]
    for _ in range(20):  # 20 kat çoğalt
        all_data = pd.concat([all_data, target_df], ignore_index=True)

print("\n Sınıf sayıları (sonra):")
print(all_data["label"].value_counts())


# Her sınıftan en az 150 örnek al
df_balanced = all_data.groupby("label").apply(lambda x: x.sample(min(150, len(x)), random_state=42)).reset_index(drop=True)

# Her sınıftan 10000 örnek al
df_filtered = all_data.groupby("label").apply(lambda x: x.sample(min(len(x), 10000), random_state=42))
df_filtered = df_filtered.reset_index(drop=True)

# Yeni sınıf dağılımını görelim
print(" Yeni sınıf dağılımı:")
print(df_filtered["label"].value_counts())

# Kaydet
df_filtered.to_csv("/content/drive/MyDrive/data/train_cleaned_balanced2.csv", index=False)



📊 Sınıf sayıları (önce):
label
öfke                 19961
normal               16360
depresyon            15414
intihar eğilimi      10652
anksiyete             8798
üzgünlük              6947
sevinç                5050
umutlu                4390
bipolar               2777
stres                 2587
kişilik bozukluğu     1077
hakaret                528
tehdit                 478
nefret söylemi          57
Name: count, dtype: int64

📈 Sınıf sayıları (sonra):
label
kişilik bozukluğu    22617
öfke                 19961
normal               16360
depresyon            15414
hakaret              11088
intihar eğilimi      10652
tehdit               10038
anksiyete             8798
üzgünlük              6947
sevinç                5050
umutlu                4390
bipolar               2777
stres                 2587
nefret söylemi          57
Name: count, dtype: int64
🧪 Yeni sınıf dağılımı:
label
depresyon            10000
hakaret              10000
intihar eğilimi      10000
kişilik bozukluğu 

##tokenization


In [ ]:
def remove_patterns(text):
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove markdown-style links
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    # Remove handles (that start with '@')
    text = re.sub(r'@\w+', '', text)
    # Remove punctuation and other special characters
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# Apply the function to the 'statement' column
all_data['text'] = all_data['text'].apply(remove_patterns)


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Apply word_tokenize to each element in the 'statement' column

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

all_data['tokens'] = all_data['text'].apply(word_tokenize)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# Initialize the stemmer
stemmer = PorterStemmer()

# Function to stem tokens and convert them to strings
def stem_tokens(tokens):
    return ' '.join(stemmer.stem(str(token)) for token in tokens)

# Apply the function to the 'tokens' column
all_data['tokens_stemmed'] = all_data['tokens'].apply(stem_tokens)


In [ ]:
# 🔹 3. Label encoding
le = LabelEncoder()
all_data["label_encoded"] = le.fit_transform(all_data["label"])
all_data.label_encoded.unique()

array([ 0,  8, 12, 13, 11, 10,  3,  6,  7,  2,  4,  9,  1,  5])

In [ ]:
all_data['num_of_characters'] = all_data['text'].apply(len)


In [ ]:
import re

def count_sentences(text):
    sentences = re.split(r'[.!?]+', text)
    sentences = [s for s in sentences if s.strip() != '']  # boşları sil
    return len(sentences)

all_data['num_of_sentences'] = all_data['text'].apply(count_sentences)


In [ ]:
X = all_data[['tokens_stemmed',	'num_of_characters',	'num_of_sentences']]
y = all_data[['label_encoded']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:

# 1. Initialize TF-IDF Vectorizer and fit/transform on the 'tokens' column
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=50000)
X_train_tfidf = vectorizer.fit_transform(X_train['tokens_stemmed'])
X_test_tfidf = vectorizer.transform(X_test['tokens_stemmed'])

# 2. Extract numerical features
X_train_num = X_train[['num_of_characters', 'num_of_sentences']].values
X_test_num = X_test[['num_of_characters', 'num_of_sentences']].values

# 3. Combine TF-IDF features with numerical features
X_train_combined = hstack([X_train_tfidf, X_train_num])
X_test_combined = hstack([X_test_tfidf, X_test_num])

print('Number of feature words: ', len(vectorizer.get_feature_names_out()))

Number of feature words:  50000


In [ ]:
X_train_combined.shape

(109388, 50002)

In [ ]:
# Apply Random Over-Sampling on the vectorized data
ros = RandomOverSampler(random_state=101)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_combined, y_train)

In [ ]:
X_train_resampled.shape

(252490, 50002)

https://www.kaggle.com/code/annastasy/mental-health-sentiment-analysis-nlp-ml/notebook faydalanılmıştır